# Table Question generation: WikiSQL dataset
In this notebook, we will see how to fine-tune and evaluate a question generation model on WikiSQL dataset.

## Configuration

We start by setting some parameters to configure the process.  Note that depending on the GPU being used you may need to tune the batch size.

In [1]:
model_name_or_path="t5-small"
modality="table"
dataset_name="wikisql"
max_len=200
target_max_len=40
output_dir="../../models/qg/wikisql_nb"
learning_rate=0.0001
num_train_epochs=2
per_device_train_batch_size=8
per_device_eval_batch_size=32
evaluation_strategy='epoch'

In [2]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    evaluation_strategy='epoch',
    learning_rate=learning_rate,
    prediction_loss_only=True,
    remove_unused_columns=False,
    )
training_args.predict_with_generate=True
training_args.remove_unused_columns = False
training_args.prediction_loss_only = False

---
## WikiSQL data
Here we load one instance of WikiSQL and visualize it. <font color='red'>This part of the code is not needed to train the model </font>

In [3]:
from datasets import load_dataset
from tabulate import tabulate

def print_wikisql_instance(train_instance):
    table = train_instance['table']
    print('Table:\n',tabulate(table['rows'], headers=table['header'], tablefmt='grid'))

    print('Question = ',train_instance['question'])
    print('SQL = ', train_instance['sql']['human_readable'])

train_instance = load_dataset('wikisql', split='train[1001:1002]')[0]
print_wikisql_instance(train_instance)

Using custom data configuration default
Reusing dataset wiki_sql (/Users/saneem/.cache/huggingface/datasets/wiki_sql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)


Table:
 +-------------------+-------+---------------+----------------+------------------+---------------------+
| Player            |   No. | Nationality   | Position       | Years for Jazz   | School/Club Team    |
+===================+=======+===============+================+==================+=====================+
| Fred Saunders     |    12 | United States | Forward        | 1977-78          | Syracuse            |
+-------------------+-------+---------------+----------------+------------------+---------------------+
| Danny Schayes     |    24 | United States | Forward-Center | 1981-83          | Syracuse            |
+-------------------+-------+---------------+----------------+------------------+---------------------+
| Carey Scurry      |    22 | United States | Forward        | 1985-88          | Long Island         |
+-------------------+-------+---------------+----------------+------------------+---------------------+
| Robert Smith      |     5 | United States | Guard     

The SQL gets converted to a string format which goes as input to generator to generate question

In [4]:
from primeqa.qg.processors.table_qg.wikisql_processor import WikiSqlDataset

data = WikiSqlDataset()
processed_data = data.preprocess_data_for_qg('train[1001:1002]')
print('Question = ', processed_data['question'][0])
print('\nInput to generator = ', processed_data['input'][0])

Using custom data configuration default
Reusing dataset wiki_sql (/Users/saneem/.cache/huggingface/datasets/wiki_sql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1392.53it/s]

Question =  Which position does John Starks play?

Input to generator =  select <<sep>> Position <<sep>> Player <<cond>> equal <<cond>> John Starks <<answer>> shooting guard <<header>> Player <<hsep>> No. <<hsep>> Nationality <<hsep>> Position <<hsep>> Years for Jazz <<hsep>> School/Club Team


---
## Loading the Model

Here we load the model based on the model_name and modality parameter set above. For WikiSQL we keep modality='table'. Other option is modality='passage'

In [5]:
from primeqa.qg.models.qg_model import QGModel

qg_model = QGModel(model_name_or_path, modality=modality)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

# Loading Data

Here we load the data

In [6]:
from primeqa.qg.processors.data_loader import QGDataLoader

qgdl = QGDataLoader(
    tokenizer=qg_model.tokenizer,
    dataset_name=dataset_name,
    input_max_len=max_len,
    target_max_len=target_max_len
    )

train_dataset = qgdl.create("train[:100]")
valid_dataset = qgdl.create("validation[:50]")

Using custom data configuration default
Reusing dataset wiki_sql (/Users/saneem/.cache/huggingface/datasets/wiki_sql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 2875.57it/s]
Parameter 'function'=<function QGDataLoader.convert_to_features at 0x7f9c234ce200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/miniconda3/envs/oneqa/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Using custom data configuration default
Reusing dataset wiki_sql (/Users/saneem/.cac

  0%|          | 0/1 [00:00<?, ?ba/s]

# Train using QGTrainer
Here we create a QG trainer with the training arguments defined above and use it to train on Wikisql training data (or any custom data following the same format)

In [8]:
from primeqa.qg.trainers.qg_trainer import QGTrainer
from primeqa.qg.metrics.generation_metrics import rouge_metrics
from primeqa.qg.utils.data_collator import T2TDataCollator
import os

compute_metrics = rouge_metrics(qg_model.tokenizer)

trainer = QGTrainer(
    model=qg_model.model,
    tokenizer = qg_model.tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    valid_dataset=valid_dataset,
    data_collator=T2TDataCollator(),
    compute_metrics=compute_metrics
    )

train_results = trainer.train()
trainer.save_model()
print(train_results.metrics)

***** Running training *****
  Num examples = 100
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 26


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,2.906801,23.251000,12.696800,21.460000,21.459600
2,No log,2.777056,24.015600,13.354900,22.792700,22.706900


***** Running Evaluation *****
  Num examples = 50
  Batch size = 32
***** Running Evaluation *****
  Num examples = 50
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ../../models/qg/wikisql_nb
Configuration saved in ../../models/qg/wikisql_nb/config.json
Model weights saved in ../../models/qg/wikisql_nb/pytorch_model.bin
tokenizer config file saved in ../../models/qg/wikisql_nb/tokenizer_config.json
Special tokens file saved in ../../models/qg/wikisql_nb/special_tokens_map.json
Copy vocab file to ../../models/qg/wikisql_nb/spiece.model


{'train_runtime': 114.8694, 'train_samples_per_second': 1.741, 'train_steps_per_second': 0.226, 'total_flos': 10573578240000.0, 'train_loss': 3.134342780480018, 'epoch': 2.0}


## Evaluation

Here we evaluate the trained model on validation set

In [9]:
metrics = trainer.evaluate()
print(metrics)

***** Running Evaluation *****
  Num examples = 50
  Batch size = 32


{'eval_loss': 2.7770562171936035, 'eval_rouge1': 24.0156, 'eval_rouge2': 13.3549, 'eval_rougeL': 22.7927, 'eval_rougeLsum': 22.7069, 'eval_runtime': 14.0715, 'eval_samples_per_second': 3.553, 'eval_steps_per_second': 0.142, 'epoch': 2.0}
